# dev

In [1]:
# constants
bucket = "k9-eth"
tbl = "transactions"

## Check AWS conf

Setup the credentials in your preferred manner.

In [2]:
%%bash
aws sts get-caller-identity

{
    "UserId": "AIDARAMT7QKYJ6DR7ITQU",
    "Account": "069566694064",
    "Arn": "arn:aws:iam::069566694064:user/k9"
}


In [3]:
%%bash -s $bucket
aws s3 ls $1

                           PRE athena-output/
                           PRE transactions/


## Check S3 connection on pyspark

Note that we will need two additional `jars` in order to access AWS S3 from Spark:

- [hadoop-aws](https://mvnrepository.com/artifact/org.apache.hadoop/hadoop-aws)
- [aws-java-sdk-bundle](https://mvnrepository.com/artifact/com.amazonaws/aws-java-sdk-bundle)

Make sure they are present in your `SPARK_HOME` jars folder.
In the case of a `pip install pyspark`, the folder should be something like `<your-python-lib-path>/site-packages/pyspark/jars`.

In [1]:
from pyspark.sql import SparkSession

# spark configuration is handled by conf/base/spark.yml
# refer to the src/hooks.py for details
spark = SparkSession.builder.getOrCreate()

In [2]:
spark.read.parquet(f"s3a://{bucket}/{tbl}").printSchema()

23/01/08 13:22:43 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
23/01/08 13:22:45 WARN DataSource: Found duplicate column(s) in the data schema and the partition schema: `date`
root
 |-- gas: long (nullable = true)
 |-- hash: string (nullable = true)
 |-- input: string (nullable = true)
 |-- nonce: long (nullable = true)
 |-- value: double (nullable = true)
 |-- block_number: long (nullable = true)
 |-- block_hash: string (nullable = true)
 |-- transaction_index: long (nullable = true)
 |-- from_address: string (nullable = true)
 |-- to_address: string (nullable = true)
 |-- gas_price: long (nullable = true)
 |-- receipt_cumulative_gas_used: long (nullable = true)
 |-- receipt_gas_used: long (nullable = true)
 |-- receipt_contract_address: string (nullable = true)
 |-- receipt_status: long (nullable = true)
 |-- receipt_effective_gas_price: long (nullable = true)
 |-- transaction_type: long (nullable = true)

## Test kedro catalog

In [1]:
catalog.list()

['transactions', 'parameters', 'params:train_fraction', 'params:random_state', 'params:target_column']

In [1]:
transactions = catalog.load("transactions")

                    INFO     Loading data from 'transactions' (AthenaQueryDataSet)...           ]8;id=625964;file:///Users/k9/.pyenv/versions/3.9.11/envs/kedro-on-aws/lib/python3.9/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=986882;file:///Users/k9/.pyenv/versions/3.9.11/envs/kedro-on-aws/lib/python3.9/site-packages/kedro/io/data_catalog.py#343\343]8;;\

                    INFO     Attempt to create external table temp_tbl_6e6cd64021 with Athena  ]8;id=878328;file:///Users/k9/kedro-on-aws/src/iris/extras/datasets/athena_dataset.py\athena_dataset.py]8;;\:]8;id=617909;file:///Users/k9/kedro-on-aws/src/iris/extras/datasets/athena_dataset.py#85\85]8;;\
                             CTAS query.                                                                           

[01/08/23 15:02:50] INFO     Athena query output files saved to:                              ]8;id=140197;file:///Users/k9/kedro-on-aws/src/iris/extras/datasets/athena_dataset.py\athena_dataset.py]8;;\:]8;id=240239;file:///Users/k9/kedro-on-aws/src/iris/extras/datasets/athena_dataset.py#109\109]8;;\
                             s3://k9-eth/athena-output/tables/8c31f53a-353e-4153-90ab-b6bcfcc                      
                             75515                                                                                 

23/01/08 15:02:50 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [2]:
transactions.printSchema()

root
 |-- date: string (nullable = true)
 |-- hash: string (nullable = true)
 |-- nonce: long (nullable = true)
 |-- transaction_index: long (nullable = true)
 |-- from_address: string (nullable = true)
 |-- to_address: string (nullable = true)
 |-- value: double (nullable = true)
 |-- gas: long (nullable = true)
 |-- gas_price: long (nullable = true)
 |-- input: string (nullable = true)
 |-- receipt_cumulative_gas_used: long (nullable = true)
 |-- receipt_gas_used: long (nullable = true)
 |-- receipt_contract_address: string (nullable = true)
 |-- receipt_status: long (nullable = true)
 |-- block_timestamp: timestamp (nullable = true)
 |-- block_number: long (nullable = true)
 |-- block_hash: string (nullable = true)
 |-- max_fee_per_gas: long (nullable = true)
 |-- max_priority_fee_per_gas: long (nullable = true)
 |-- transaction_type: long (nullable = true)
 |-- receipt_effective_gas_price: long (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = t

In [4]:
transactions.count()

100